# Query Text Data workflow with "Lazy GraphRAG" retrieval mechanism

Vector-based semantic search is a "best first" search approach for relevant chunks - it simply takes the top k most similar chunks based on embedding distances. It has no sense of the breadth of semantic content in the dataset.

Full [GraphRAG](https://aka.ms/graphrag) is a "breadth first" search approach for relevant content in community summaries - it always queries each community. It has no sense of which communities are best.

Lazy GraphRAG is a hybrid approach that requires no summarization-based index (which is expensive, especially if the user only has a few queries). Instead, it relies purely on an embedding-based index for input text chunks (cheap, fast) augmented by the concept graph of noun-phrase cooccurrences extracted from these chunks (free, fast). It works as follows:

- Given a query, text chunks are ranked using vector-based semantic search.
- Chunk rankings are used to determine community rankings in the concept graph.
- In community rank order, the top k chunks are subject to a rapid relevant test (cheap, fast). If relevant chunks are found, the community is retained, otherwise it is discarded.
- After n communities fail to return any relevant chunks, the "iterative narrowing" process repeats with remaining communities in rank order.
- The process terminates when no more testable chunks exist or a predefined testing budget is reached (e.g., 100 relevance tests).

The advantages of this approach include:

- A balance of best-first (semantic search) and breadth-first (community structure) approaches for any testing budget
- Very fast and almost free - suitable for one-off and exploratory queries before committing to a full indexing
- Easily accommodates dynamic contexts where new texts/chunks are streaming in to the dataset (supported in the current implementation)
- Provides a consistent level of "effort" for a fixed test budget, providing various standards of answers to questions (e.g., global questions) for which there is no ground truth (e.g., 1000 chunks = gold standard, 500 chunks = silver standard, 100 chunks = bronze standard, etc.)


In [1]:
import sys

sys.path.append("..")

In [2]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU' # Avoids threadpoolctl error in Linux and MacOS
from toolkit.AI.openai_configuration import OpenAIConfiguration
from toolkit.AI.openai_embedder import OpenAIEmbedder
from toolkit.helpers.constants import CACHE_PATH

import toolkit.query_text_data.input_processor as input_processor
import toolkit.query_text_data.relevance_assessor as relevance_assessor
import toolkit.query_text_data.answer_builder as answer_builder
import toolkit.query_text_data.graph_builder as graph_builder
import toolkit.query_text_data.pattern_detector as pattern_detector
import toolkit.query_text_data.helper_functions as helper_functions
import nest_asyncio # Necessary to run async code in ipynb

nest_asyncio.apply()

c:\Users\daedge\AppData\Local\pypoetry\Cache\virtualenvs\intelligence-toolkit-lXFNld9n-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
# Comment/uncomment the following lines to toggle access to library code updates
import importlib
importlib.reload(input_processor)
importlib.reload(pattern_detector)
importlib.reload(helper_functions)
importlib.reload(relevance_assessor)
importlib.reload(answer_builder)
importlib.reload(graph_builder)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


<module 'toolkit.query_text_data.graph_builder' from 'c:\\Users\\daedge\\Code\\ITK\\intelligence-toolkit\\example_notebooks\\..\\toolkit\\query_text_data\\graph_builder.py'>

In [4]:
# Set up the AI model and embedding model
ai_configuration = OpenAIConfiguration(
    {
        "api_type": "OpenAI",
        "api_key": os.environ["OPENAI_API_KEY"],
        "model": "gpt-4o-2024-08-06",
    }
)

text_embedder = OpenAIEmbedder(
    configuration=ai_configuration,
)

In [5]:
# Provide text inputs as rows of a dataframe
input_path = "../example_outputs/query_text_data/news_articles/news_articles_texts.csv"
file_name = input_path.split("/")[-1]
file_to_chunks = input_processor.process_file_bytes({file_name: open(input_path, 'rb').read()}, input_processor.PeriodOption.NONE)
print("Chunked input text")

Chunked input text


In [6]:
max_cluster_size = 25 # The maximum number of concepts in a leaf-level community in the concept graph
# Process the chunks into the index data structures
(
    cid_to_text,
    text_to_cid,
    period_concept_graphs,
    hierarchical_communities,
    community_to_label,
    concept_to_cids,
    cid_to_concepts,
    previous_cid,
    next_cid,
    period_to_cids,
    node_period_counts,
    edge_period_counts,
) = input_processor.process_chunks(file_to_chunks=file_to_chunks, max_cluster_size=max_cluster_size)
print(f"Processed chunks")

Processed chunks


In [7]:
# Embed the text chunks
cid_to_vector = await helper_functions.embed_texts(
    cid_to_text=cid_to_text,
    text_embedder=text_embedder,
)
print(f"Embedded chunk text")

Got 501 existing texts
Got 0 new texts
Embedded chunk text


In [8]:
question = "What events are discussed?"
relevance_test_budget = 20 # How many relevance tests are permitted per query. Higher values may provide higher quality results at higher cost
adjacent_search_steps = 1 # How many chunks before and after each relevant chunk to test, once the relevance test budget is near or the search process has terminted
community_relevance_tests = 5 # How many relevance tests to run on each community in turn
relevance_test_batch_size = 5 # How many relevance tests to run in parallel at a time
community_ranking_chunks = 5 # How many chunks to use to rank communities by relevance
irrelevant_community_restart = 3 # When to restart testing communities in relevance order

# Mine relevant chunks to the question
relevant_cids, chunk_progress = await relevance_assessor.detect_relevant_chunks(
        ai_configuration=ai_configuration,
        question=question,
        cid_to_text=cid_to_text,
        cid_to_concepts=cid_to_concepts,
        cid_to_vector=cid_to_vector,
        hierarchical_communities=hierarchical_communities,
        community_to_label=community_to_label,
        previous_cid=previous_cid,
        next_cid=next_cid,
        embedder=text_embedder,
        embedding_cache=CACHE_PATH,
        select_logit_bias=5,
        adjacent_search_steps=adjacent_search_steps,
        community_ranking_chunks=community_ranking_chunks,
        relevance_test_budget=relevance_test_budget,
        community_relevance_tests=community_relevance_tests,
        relevance_test_batch_size=relevance_test_batch_size,
        irrelevant_community_restart=irrelevant_community_restart,
    )
print(f"Mined relevant chunks")
    


Semantic search cids: [213, 104, 50, 38, 449, 76, 178, 182, 29, 359, 344, 484, 376, 314, 464, 381, 399, 181, 93, 324, 179, 150, 202, 409, 144, 155, 441, 96, 279, 64, 229, 237, 56, 189, 55, 364, 370, 219, 251, 400, 388, 84, 62, 341, 284, 130, 47, 485, 277, 31, 183, 310, 196, 239, 267, 475, 290, 470, 457, 138, 440, 112, 296, 446, 164, 180, 24, 418, 224, 493, 4, 468, 168, 173, 380, 244, 188, 90, 233, 382, 65, 349, 442, 348, 14, 304, 109, 337, 105, 379, 391, 408, 166, 443, 165, 216, 72, 374, 20, 152]
Level 0 community sequence: ['1.1', '1.6', '1.10', '1.7', '1.3', '1.4', '1.14', '1.8', '1.18', '1.5', '1.12', '1.11', '1.9', '1.17', '1.19', '1.2', '1.16', '1.13', '1.15']
Level 1 community sequence: ['1.1.1', '1.1.5', '1.1.2', '1.1.4', '1.10.1', '1.1.3', '1.6.3', '1.6.2', '1.1.6', '1.6.4', '1.6.1', '1.18', '1.14.2', '1.3.2', '1.3.3', '1.10.2', '1.1.7', '1.1.8', '1.14.4', '1.4.1', '1.5.2', '1.8.4', '1.10.3', '1.3.4', '1.9.1', '1.17.3', '1.12.5', '1.10.5', '1.7.25', '1.7.12', '1.4.3', '1.11.2',

100%|██████████| 5/5 [00:01<00:00,  4.13it/s]


Batch 1 of 1: 5 relevant chunks
Community 1 relevant? True
Incrementing level
New level 0 loop after 5 tests
Community sequence: ['1.1', '1.6', '1.10', '1.7', '1.3', '1.4', '1.14', '1.8', '1.18', '1.5', '1.12', '1.11', '1.9', '1.17', '1.19', '1.2', '1.16', '1.13', '1.15']
Assessing relevance for community 1.1 with chunks [178, 182, 29, 359, 344]
Assessing relevance for topic 1.1 with 5 chunks


100%|██████████| 5/5 [00:01<00:00,  4.31it/s]


Batch 1 of 1: 5 relevant chunks
Community 1.1 relevant? True
Assessing relevance for community 1.6 with chunks [76, 93, 184, 261, 458]
Assessing relevance for topic 1.6 with 5 chunks


...

Batch 1 of 1: 0 relevant chunks
Community 1.7 relevant? False
Assessing relevance for community 1.3 with chunks [47, 267, 468, 188, 408]
Assessing relevance for topic 1.3 with 5 chunks


0it [00:00, ?it/s]


Batch 1 of 1: 0 relevant chunks
Community 1.3 relevant? False
Assessing relevance for community 1.4 with chunks [65, 107, 35, 0, 404]
Assessing relevance for topic 1.4 with 5 chunks


0it [00:00, ?it/s]

Batch 1 of 1: 0 relevant chunks
Community 1.4 relevant? False
0 successive irrelevant communities; restarting
Incrementing level
Assessing relevance for neighbours with 0 chunks
Mined relevant chunks


In [9]:
extract_claims = True # Whether to extract claims from relevant text chunks
search_depth = 10 # If extracting claims, how many chunks to search for additional claim support
target_chunks_per_cluster = 5 # How many chunks to aim to analyze together in a single LLM call

# Generate an extended answer to the question, which could then be summarized
(
    extended_answer,
    references,
    referenced_chunks,
    net_new_sources,
) = await answer_builder.answer_question(
    ai_configuration=ai_configuration,
    question=question,
    relevant_cids=relevant_cids,
    cid_to_text=cid_to_text,
    cid_to_vector=cid_to_vector,
    target_chunks_per_cluster=target_chunks_per_cluster,
    embedder=text_embedder,
    embedding_cache=CACHE_PATH,
    extract_claims=extract_claims,
    search_depth=search_depth
)
print(f"Answered question")

c:\Users\daedge\AppData\Local\pypoetry\Cache\virtualenvs\intelligence-toolkit-lXFNld9n-py3.11\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████| 15/15 [00:12<00:00,  1.21it/s]


Answered question


In [10]:
# Output the final extended answer
print(extended_answer)

# Global Events Highlight the Importance of Cooperation and Innovation in Addressing Climate, Economic, and Cultural Challenges

*In response to: What events are discussed?*

## Executive summary

The report explores a range of global events that emphasize the need for international cooperation, innovative strategies, and cultural celebration. These events include climate change summits, economic forums, wellness retreats, and cultural festivals, each addressing different aspects of global challenges and opportunities. The themes of immediate action on climate change, economic resilience, and cultural diversity are recurrent, showcasing a global commitment to sustainable development and community engagement.

## Theme: Immediate Action and International Cooperation are Essential to Combat Climate Change

Multiple international conferences and summits have consistently emphasized the urgent need for immediate action and international cooperation to address climate change. These events h